#### Notes and plots

Plot sources(figure 1 to 8)

https://i.ibb.co/GvZXHZLs/figure1.png

https://i.ibb.co/8LRCHrF2/figure2.png

https://i.ibb.co/Jw7wWQtg/figure3.png

https://i.ibb.co/t191Cng/figure4.png

https://i.ibb.co/yJbpvvT/figure5.png

https://i.ibb.co/vv12dTTR/figure6.png

https://i.ibb.co/nsMmYRfK/figure7.png

https://i.ibb.co/chN8Q9pg/figure8.png

# **Introduction**

Our project is PCA and Shape mode analysis. We are given **"fluidE.mat** file, which contains

**XX** and **YY**: Matrices of x-coordinate and y-coordinate of each point along the flagellum at every frame.The shapes of the falgellu are encoded in these two matrices.

**space_scale**: Distance between adjacent tracked points in micrometers.

**time_scape**: Time between video frames in seconds.


Togetheter we can use these datas to understand the flagellar movement pattern(how flagellum shapes change over time)





# **Part (1)**

Our first instruction is to visualize the  **raw** flagellum shapes changes over time.


We first output the Dimensions:  $m$, the number spatial points, $n$ the number of time steps(points)

>disp('Data:');

>disp(['Number of spatial points: ', num2str(m)]);

>disp(['Number of time points: ', num2str(n)]);

>disp(['Space scale (\mum): ', num2str(space_scale)]);

>disp(['Time scale (seconds): ', num2str(time_scale)])

Then  we plot the first 5 time steps. (note, we refer figure2 as 1 for explanation purpose).

<img src="https://i.ibb.co/8LRCHrF2/figure2.png" alt="figure2" width="900"/>

 We can also display the duration of the first 5 time steps by doing

 >disp(['Duration of the first 5 time steps: ', num2str(4 * time_scale), ' seconds']);

 And it outputs:

 > Duration of the first 5 time steps: 0.0066007 seconds


This and the figure shows how the flagellum's shape changes over the first 5 frames in 6.6 ms. Each plotted curves represent 5 continuous or successive snapshots(time steps) of the flagellum shape in 6.6 milliseconds.


Next we plot all 80 time steps of the flagellum shapes.

<img src="https://i.ibb.co/GvZXHZLs/figure1.png" alt="figure2" width="900"/>


#### **Creating figures in matlab:**

At any single frame (time $t$), the two vectors

$$
XX(:,i) = \bigl[x_1, x_2, \dots, x_m\bigr]^T
\quad\text{and}\quad
YY(:,i) = \bigl[y_1, y_2, \dots, y_m\bigr]^T
$$

Give you $m$ discrete $(x_j,y_j)$ points along the flagellum. By plotting the sequence of points for a specific time step, and then drawing lines connecting them. This produces the curvy lines that represent the flagellum shape at time $t_i$.

To do this in matlab we use this command

> plot(XX(:,i), YY(:,i))

This do what we described above, it takes sequence of points $(x_1,y_1),\;(x_2,y_2),\;\dots,\;(x_m,y_m)$  and draw straight line segments from $(x_j,y_j)$ to $(x_{j+1},y_{j+1})$ for $j=1\ldots m-1$.

Then we simply create 2 loops, one that loop through every (80) time steps. And one that loop through 5 time steps.


# **Part (2)**

We want to quantify how the flagellum bends and propagates waves along its length. So we need to convert the shape into tangent angles, which describes the local direction of the flagellum at each point along its length.

#### **Steps in matlab**

For each pair of adjacent points $(x_j, y_j)$ and $(x_{j+1}, y_{j+1})$ at a given time step $t_i$:

We calculate the slope of these two adjacent points: $$ \frac{\Delta y_j (t_i)}{\Delta x_j (t_i) } = \frac{y_{j+1} (t_i) - y_j (t_i) }{x_{j+1}(t_i) - x_j (t_i)}$$.

Then convert them into angle using atan2 which give us the tangent angle $ \phi_j(t_i)$.

$$
\phi_j(t_i) = \mathrm{atan2}\bigl(\Delta y_j,\Delta x_j\bigr)\in(-\pi,\pi]. \color{grey} { \; \equiv \; arctan\left( \frac{\Delta y_j (t_i) }{\Delta x_j (t_i) } \right)}
$$

We repeat the previous steps for every time step $t_i$ for $i = 1,..,80$  and build the $(m-1)\times n$ matrix $\Phi$

$$
  \Phi =
  \begin{bmatrix}
    \phi_1(t_1) & \phi_1(t_2) & \cdots & \phi_1(t_n) \\
    \phi_2(t_1) & \phi_2(t_2) & \cdots & \phi_2(t_n) \\
    \vdots      & \vdots      & \ddots & \vdots      \\
    \phi_{m-1}(t_1) & \phi_{m-1}(t_2) & \cdots & \phi_{m-1}(t_n)
  \end{bmatrix},
$$

with enties

$$
  \Phi_{j,i} = \phi_j(t_i).
$$

Because $\phi_j \in (-\pi,\pi]$, we use unwrap on tangent angles  function to remove “jump” by $\pm2\pi$.

Next we need to create physical axes for the kymograph.

**X-axis:** Spatial (arclength) axis:

$$s_j = (j-1) \cdot \text{space_scale}$$

**Y-axis:** Temporal axis:

$$t_i=(i-1) \cdot \text{time_scale}$$

This tell us where each tangent angle measurement occurred( in micrometers ) along the length of the flagellum, and when.


This whole process is done in matlab as:

```
phi = zeros(m-1, n);
for i = 1:80
    dx = diff(XX(:,i));  % [x2-x1, ..., x32-x31] at time i
    dy = diff(YY(:,i));
    phi(:,i) = atan2(dy, dx);  % Store column i
end

phi_unwrapped = zeros(size(phi));
for i = 1:80
    phi_unwrapped(:,i) = unwrap(phi(:,i));
end

delta_s = space_scale; % Distance between two adjacent points
delta_t = time_scale;  % Time between frames

%s = [0*space_scale, 1*space_scale, 2*space_scale, ...]
s = (0:size(phi_unwrapped,1)-1)' * delta_s; % Position where phi is defined.
t = (0:n-1)' * delta_t; % When phi is calculated                    

```

<img src="https://i.ibb.co/Jw7wWQtg/figure3.png" alt="figure2" width="900"/>


**Intepretation of kymograph**

At a fixed time $t_i$, the  horizontal line across the kymograph at that $t_i$, show a sequence of colours varying with $s$. Each colour at position $s_j$ on that line is the tangent angle $\phi(s_j,t_i)$.

But this doesn't show  full physical shape at a given at given $t_i$
  



#**Part (3):**

In part 2 we create tangent angles matrix $\Phi$, in this part We demean the tangent matrix $\Phi$ and perform SVD on it.

```
phi_mean = mean(phi_unwrapped, 2);
demean_phi = phi_unwrapped - phi_mean;
[U,S,V] = svd(demean_phi, 'econ');
```

First we compute `phi_mean` ($\bar\phi_j$), the average of the tangent angle at each spatial points across all $n$ time steps/frame.

$$
  \bar\phi_j
  = \frac{1}{n}\sum_{i=1}^n \Phi_{j,i},
  \quad j=1,\dots,m-1.
$$

Then we subtract it from every column of the data matrix to get the demeaned matrix `demean_phi` ($\Phi'$)

$$
\Phi' =\Phi_{j,i} - \bar\phi_j
\qquad
\Phi'\in\mathbb R^{(m-1)\times n}.
$$

Next we perform **SVD** on $\Phi'$.

$$
\Phi' = U\,S\,V^T,
$$

Where

$U \in \mathbb{R}^{(m-1) \times (m-1)} $: **Left singular vectors** (columns are **shape modes** $ \mathbf{u}_k $)

$\Sigma \in \mathbb{R}^{(m-1) \times n} $: Diagonal matrix of **singular values** $ \sigma_k = s_k \geq 0 $

$V \in \mathbb{R}^{n \times n} $: **Right singular vectors** (columns are **temporal modes** $ \mathbf{v}_k $)

We then we create 3 plots each of these matrices.

#### **The first 4 shape modes(figure 4)**

```
figure; clf; hold on;
for k = 1:4
    plot(U(:,k))
end
title('First 4 Shape modes');
hold off;
```
<img src="https://i.ibb.co/Vc3nvfq9/figure4.png" alt="figure4" width="1200"/>

Each lines(shape modes) correspond to a singular (column) vector $\mathbf(u_i)$ from $U$.

 **X-axis**: Spatial position along the flagellum, from base ($ j=1 $) to tip ($ j=m-1 $).

**Y-axis**: The bending amplitude (in radians) of mode $k$ at point $j$.

This figure how the flagellum bends spatially for each of the first four modes.


#### **The first 4 temporal modes**

```
figure; clf; hold on;
for k = 1:4
    plot(V(:,k))
end
title('First 4 Temporal Modes');
hold off;
```
<img src="https://i.ibb.co/yJbpvvT/figure5.png" alt="figure5" width="1200"/>


**X-axis**: Time $t_i$ for $ i = 1,\dots,n $

**Y-axis**: Mode coefficient $\mathbf{v_k(t_i)}$ with unit length.


#### **The strength of the modes (Singular values): Variance Retained vs. Number of Modes**

We compute the cumulative fraction of total variance explained by the first  $k$ modes:


$$
\frac{\sum_{k=1}^K s_k^2}{\sum_{k=1}^r s_k^2} \qquad \text{ for k = 1,....,m -1}
$$

Where $s_k^2 = \sigma_k^2$ is the squared singular values.

```
s = diag(S);
figure;
plot(cumsum(s.^2)./sum(s.^2), 'bo', 'MarkerFaceColor','b','MarkerSize',6)
xlabel("kth Singular Value");  
ylabel("% Information Retained");
title("Strength of Singular Values")

```

<img src="https://i.ibb.co/vv12dTTR/figure6.png" alt="figure6" width="1200"/>

**X-axis**: $k$ amount of modes.

**Y-axis**: Fraction of total variance retained

THis figures show that the first two modes $u_1, u_2$ captures most of the flagellum's shape variance.

# Part (5)


Using the least squares fitted  $V_1 $ and $ V_2$, we can reconstruct the tangent angle matrix using the first two shapes mode.


$$
\phi_{j,i}
= \bar\phi_j + V_{1,\rm fit}(i)\,u_{1,j} + V_{2,\rm fit}(i)\,u_{2,j},
$$


To reconstruct  $(x,y)$ coordinates by doing at each point $x_j$ we compute

$$
\left[\begin{array}{l}
x_{j} \\
y_{j}
\end{array}\right]=\Delta s \sum_{i=1}^{j}\left[\begin{array}{c}
\cos \left(\phi_{i}\right) \\
\sin \left(\phi_{i}\right)
\end{array}\right]
$$



<img src="https://i.ibb.co/chN8Q9pg/figure8.png" alt="figure6" width="1200"/>




```
%% Part 5:
phi_recon = zeros(size(phi_unwrapped));
for i = 1:size(phi_unwrapped, 2)
    phi_recon(:,i) = phi_mean + V_1_fit(i) * U(:,1) + V_2_fit(i) * U(:,2);
end

X_recon = zeros(m, n);
Y_recon = zeros(m, n);
figure;
for i = 1:10
    % Original positions
    x_og = zeros(m, 1);
    y_og = zeros(m, 1);
    for j = 2:m
        x_og(j) = x_og(j-1) + space_scale * cos(phi_unwrapped(j-1, i));
        y_og(j) = y_og(j-1) + space_scale * sin(phi_unwrapped(j-1, i));
    end
    
    % Reconstructed positions
    x_recon = zeros(m, 1);
    y_recon = zeros(m, 1);
    for j = 2:m
        x_recon(j) = x_recon(j-1) + space_scale * cos(phi_recon(j-1, i));
        y_recon(j) = y_recon(j-1) + space_scale * sin(phi_recon(j-1, i));
    end
    
    plot(x_og, y_og, 'b-');
    hold on;
    plot(x_recon, y_recon, 'r--');
end
hold off;
xlabel('X');
ylabel('Y');
title('Original vs Reconstructed Shapes');
legend('Original', 'Reconstructed');
axis equal;
grid on;
```